In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from glob import glob

In [2]:
images_path = '../input/flickr8ksau/flickr8k-sau/Flickr_Data/Images/'
images = glob(images_path+'*.jpg')
len(images)

In [3]:
from keras.applications.resnet import ResNet50
incept_model = ResNet50(include_top=True)

In [4]:
from keras.models import Model
last = incept_model.layers[-2].output
modele = Model(inputs = incept_model.input,outputs = last)
#modele.summary()

In [5]:
images_features = {}
count = 0
for i in images:
    img = cv2.imread(i)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(224,224))
    img = img.reshape(1,224,224,3)
    pred = modele.predict(img).reshape(2048,)
    img_name = i.split('/')[-1]
    images_features[img_name] = pred
    count += 1
    if count >= 1500:
        break

In [6]:
caption_path = '../input/flickr8ksau/Flickr_Data/Flickr_TextData/Flickr8k.token.txt'

In [7]:
captions = open(caption_path,'rb').read().decode('utf-8').split('\n')

In [8]:
captions_dict = {}
for i in captions:
    try:
        img_name = i.split('\t')[0][:-2]
        caption = i.split('\t')[1]
        if img_name in images_features:
            if img_name not in captions_dict:
                captions_dict[img_name]=[caption]
            else:
                captions_dict[img_name].append(caption)
    except:
        pass

In [9]:
def preprocessed(txt):
    modified = txt.lower()
    modified = 'startofseq ' + modified + ' endofseq'
    return modified

In [10]:
for k,v in captions_dict.items():
    for vv in v:
        captions_dict[k][v.index(vv)] = preprocessed(vv)

In [11]:
count_words = {}
count = 1
for k,vv in captions_dict.items():
    for v in vv:
        for word in v.split():
            if word not in count_words:
                count_words[word] = count
                count += 1

In [12]:
for k,vv in captions_dict.items():
    for v in vv:
        encoded = []
        for word in v.split():
            encoded.append(count_words[word])
        captions_dict[k][vv.index(v)] = encoded

In [13]:
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [14]:
MAX_LEN = 0
for k,vv in captions_dict.items():
    for v in vv:
        if len(v) > MAX_LEN:
            MAX_LEN = len(v)
            print(v)

In [15]:
vocab_size = len(count_words)
def generator(photo,caption):
    n_samples = 0
    X = []
    y_in = []
    y_out = []
    for k,vv in caption.items():
        for v in vv:
            for i in range(1,len(v)):
                X.append(photo[k])
                
                in_seq = [v[:i]]
                out_seq = v[i]
                
                in_seq = pad_sequences(in_seq,maxlen = MAX_LEN,padding = 'post',truncating = 'post')[0]
                out_seq = to_categorical([out_seq],num_classes=vocab_size+1)[0]
                
                y_in.append(in_seq)
                y_out.append(out_seq)
                
    return X,y_in,y_out 

In [16]:
X,y_in,y_out = generator(images_features,captions_dict)

In [17]:
type(y_in)

In [18]:
X = np.array(X)
y_in = np.array(y_in,dtype = 'float64')
y_out = np.array(y_out,dtype = 'float64')

In [19]:
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM,GRU
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from tqdm import tqdm

In [23]:
embedding_size = 128
max_len = MAX_LEN
vocab_size = len(count_words) + 1
image_model = Sequential()
image_model.add(Dense(embedding_size,input_shape = (2048,),activation = 'relu'))
image_model.add(RepeatVector(max_len))
image_model.summary()

language_model = Sequential()
language_model.add(Embedding(input_dim = vocab_size,output_dim = embedding_size,input_length = max_len))
language_model.add(LSTM(256,return_sequences = True))
language_model.add(TimeDistributed(Dense(embedding_size)))
language_model.summary()

conca = Concatenate()([image_model.output,language_model.output])

x = LSTM(128,return_sequences = True)(conca)
x = LSTM(512,return_sequences = False)(x)
x = Dense(vocab_size)(x)

out = Activation('softmax')(x)

model = Model(inputs = [image_model.input,language_model.input],outputs = out)
model.compile(loss = 'categorical_crossentropy',optimizer = 'RMSProp',metrics = ['accuracy'])
model.summary()

In [24]:
model.fit([X,y_in],y_out,batch_size = 512,epochs = 500)

In [25]:
inv_dict = {v:k for k,v in count_words.items()}

In [26]:
def get_image(x):
    test_img_path = images[x]
    test_img = cv2.imread(test_img_path)
    test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2RGB)
    test_img = cv2.resize(test_img,(224,224))
    test_img = np.reshape(test_img,(1,224,224,3))
    return test_img

In [27]:
test_feature = modele.predict(get_image(2500)).reshape(1,2048)

In [32]:
import matplotlib.pyplot as plt

for i in range(5):
    
    no = np.random.randint(1500,7000,(1,1))[0,0]
    test_feature = modele.predict(get_image(no)).reshape(1,2048)
    
    test_img_path = images[no]
    test_img = cv2.imread(test_img_path)
    test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)


    text_inp = ['startofseq']

    count = 0
    caption = ''
    while count < 25:
        count += 1

        encoded = []
        for i in text_inp:
            encoded.append(count_words[i])

        encoded = [encoded]

        encoded = pad_sequences(encoded, padding='post', truncating='post', maxlen=MAX_LEN)


        prediction = np.argmax(model.predict([test_feature, encoded]))

        sampled_word = inv_dict[prediction]

        caption = caption + ' ' + sampled_word
            
        if sampled_word == 'endofseq':
            break

        text_inp.append(sampled_word)
        
    plt.figure()
    plt.imshow(test_img)
    plt.xlabel(caption)

In [33]:
model.save('model.h5')
model.save_weights('mine_model_weights.h5')
np.save('vocab.npy', count_words)

In [34]:
from IPython.display import FileLink
FileLink(r'mine_model_weights.h5')